<a href="https://colab.research.google.com/github/camiloseguel-alt/Speed-Public/blob/main/Share_PT_%26_Geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar los paquetes necesarios
%pip install --quiet google-cloud-bigquery
!pip install --quiet google-cloud-bigquery pandas openpyxl

from google.cloud import bigquery
from google.colab import auth

# Importar las librerías necesarias
from google.cloud import bigquery
import pandas as pd
from datetime import datetime

# Authenticate to Google Cloud
auth.authenticate_user()

# Conectar al cliente de BigQuery
# Construct a BigQuery client object.
# TODO(developer): Set default project.
# client = bigquery.Client(project="your-project-id")
client = bigquery.Client(project="meli-bi-data")


# Obtener información de la tabla
table_ref = client.dataset("WHOWNER", project="meli-bi-data").table("LK_SHP_SHIPMENTS_HISTORICAL_ND")
table = client.get_table(table_ref)

# Función para mostrar el menú de selección
def menu_selection():
    while True:
        print("\n**********************************************************************")
        print("POR FAVOR, PROPORCIONE LOS SIGUIENTES PARÁMETROS PARA OBTENER LA INFORMACIÓN HISTORICA/REAL:")
        print("**********************************************************************")
        print("\nSELECCIONE EL PAÍS (SIT_SITE_ID):")
        site_options = ['MCO', 'MLC', 'MLA', 'MPE', 'MLU', 'MLM', 'MLB', 'MEC']
        site_options.sort() # Sort the list alphabetically
        for idx, option in enumerate(site_options):
            print(f"{idx + 1}. {option}")

        while True:
            try:
                country_index = int(input("Ingrese el número del país: ")) - 1
                if 0 <= country_index < len(site_options):
                    country = site_options[country_index]
                    break
                else:
                    print("Número de país inválido. Inténtalo de nuevo.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número.")


        print("\nSELECCIONE EL TIPO DE INFORMACIÓN:")
        info_types = ['First Visit', 'Handling', 'Created']
        for idx, option in enumerate(info_types):
            print(f"{idx + 1}. {option}")

        while True:
            try:
                info_type_index = int(input("Ingrese el número del tipo de información: ")) - 1
                if 0 <= info_type_index < len(info_types):
                    info_type = info_types[info_type_index]
                    break
                else:
                    print("Número de tipo de información inválido. Inténtalo de nuevo.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número.")

        # Año
        while True:
            try:
                year = int(input(f"\nINGRESE EL AÑO (2023 - {datetime.now().year}): "))
                if 2023 <= year <= datetime.now().year:
                    break
                else:
                    print("Fecha inválida, debe ser entre 2023 y el año actual. Inténtalo de nuevo.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número.")


        # Mes
        while True:
            try:
                month = int(input(f"\nINGRESE EL MES (1 - 12): "))
                if 1 <= month <= 12:
                    if year == datetime.now().year and month > datetime.now().month:
                         print("Mes inválido para el año actual. Inténtalo de nuevo.")
                    else:
                        break
                else:
                    print("Mes inválido, debe ser entre 1 y 12. Inténtalo de nuevo.")
            except ValueError:
                print("Entrada inválida. Por favor, ingrese un número.")


        return country, info_type, year, month

# Función para generar y ejecutar la consulta
def execute_query(country, info_type, year, month):
    month_str = f"{month:02}"

    date_column_map = {
        'First Visit': 'SHP_DATE_FIRST_VISIT_ID_LTZ',
        'Handling': 'SHP_DATE_HANDLING_ID_LTZ',
        'Created': 'SHP_DATE_CREATED_ID'
    }

    date_column = date_column_map[info_type]

    print(f"Executing query with:")
    print(f"  Country: {country}")
    print(f"  Info Type: {info_type}")
    print(f"  Date Column: {date_column}")
    print(f"  Year: {year}")
    print(f"  Month: {month}")


    query = f"""
    SELECT
        SIT_SITE_ID,
        COUNT(SHP_SHIPMENT_ID) AS SHP,
        SUM (SHP_QUANTITY) AS SIS,
        CASE
            WHEN SIT_SITE_ID = 'MCO' AND SHP_ADD_STATE_ID_BUYER = 'CO-DC' THEN 'CO-BDC'
            ELSE SHP_ADD_STATE_ID_BUYER
        END AS SHP_ADD_STATE_ID_BUYER,
        CASE
            WHEN SIT_SITE_ID = 'MCO' AND SHP_ADD_STATE_ID_SELLER = 'CO-DC' THEN 'CO-BDC'
            ELSE SHP_ADD_STATE_ID_SELLER
        END AS SHP_ADD_STATE_ID_SELLER,
        EXTRACT(YEAR FROM {date_column}) AS YEAR,
        EXTRACT(MONTH FROM {date_column}) AS MONTH,
        CASE
            WHEN SHP_PICKING_TYPE_ID = 'drop_off' THEN 'DS'
            WHEN SHP_PICKING_TYPE_ID = 'xd_drop_off' THEN 'XD'
            WHEN SHP_PICKING_TYPE_ID = 'fulfillment' THEN 'FUL'
            WHEN SHP_PICKING_TYPE_ID = 'cross_docking' THEN 'XD'
            WHEN SHP_PICKING_TYPE_ID = 'self_service' THEN 'FLEX'
            ELSE NULL
        END AS SHP_PICKING_TYPE_ID,
        CASE
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 0 THEN 'SD'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 1 THEN 'ND'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 2 THEN '2D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 3 THEN '3D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 4 THEN '4D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 5 THEN '5D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 6 THEN '6D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 7 THEN '7D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 8 THEN '8D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS = 9 THEN '9D'
            WHEN SHP_LEAD_TIME_NATURAL_DAYS > 9 THEN '9DP'
            ELSE NULL
        END AS SHP_LEAD_TIME_NATURAL_DAYS
    FROM `meli-bi-data.WHOWNER.LK_SHP_SHIPMENTS_HISTORICAL_ND`
    WHERE
        SIT_SITE_ID = '{country}' AND
        LOWER(SHP_FLOW_TYPE) = 'forward' AND
        SHP_SOURCE_ID = 'MELI' AND
        EXTRACT(YEAR FROM {date_column}) = {year} AND
        EXTRACT(MONTH FROM {date_column}) = {month} AND
        SHP_LEAD_TIME_NATURAL_DAYS IS NOT NULL
    GROUP BY
        SIT_SITE_ID,
        SHP_ADD_STATE_ID_BUYER,
        SHP_ADD_STATE_ID_SELLER,
        YEAR,
        MONTH,
        SHP_PICKING_TYPE_ID,
        SHP_LEAD_TIME_NATURAL_DAYS
    """

    df = client.query(query).result().to_dataframe()

    print(f"Number of rows returned by the query: {len(df)}")
    print(f"Data type of 'SIS' column: {df['SIS'].dtype}")
    # print("\nDataFrame columns after fetching from BigQuery:")
    # print(df.columns)
    # Remove the display of the first 5 rows after fetching from BigQuery
    # display(df.head())


    # Rename Year and Month columns based on info_type
    rename_map = {}
    if info_type == 'First Visit':
        rename_map['YEAR'] = 'YEAR_FV'
        rename_map['MONTH'] = 'MONTH_FV'
        year_col = 'YEAR_FV'
        month_col = 'MONTH_FV'
    elif info_type == 'Handling':
        rename_map['YEAR'] = 'YEAR_HT'
        rename_map['MONTH'] = 'MONTH_HT'
        year_col = 'YEAR_HT'
        month_col = 'MONTH_HT'
    elif info_type == 'Created':
        rename_map['YEAR'] = 'YEAR_C'
        rename_map['MONTH'] = 'MONTH_C'
        year_col = 'YEAR_C'
        month_col = 'MONTH_C'


    df.rename(columns=rename_map, inplace=True)

    # Explicitly convert 'SIS' to numeric, coercing errors will turn unparseable values into NaN
    df['SIS'] = pd.to_numeric(df['SIS'], errors='coerce')


    df = df[df['SHP_PICKING_TYPE_ID'].notnull()]  # Ignorar no estándar

    # Print unique values in SHP_LEAD_TIME_NATURAL_DAYS before standardization to debug
    # print("Unique values in SHP_LEAD_TIME_NATURAL_DAYS before standardization:")
    # The SHP_LEAD_TIME_NATURAL_DAYS column is now the result of the CASE statement
    # print(df['SHP_LEAD_TIME_NATURAL_DAYS'].unique())


    # Calculate column SHARE_PT (J) based on the provided formula
    # SUMIFS($H:$H;$A:$A;$A2;$B:$B;$B2;$C:$C;$C2;$D:$D;$D2;$E:$E;$E2;$F:$F;$F2;$G:$G;$G2)/SUMIFS($H:$H;$A:$A;$A2;$B:$B;$B2;$C:$C;$C2)
    # H -> SHP, A -> SIT_SITE_ID, B -> YEAR_FV/HT/C, C -> MONTH_FV/HT/C, D -> SHP_PICKING_TYPE_ID, E -> SHP_ADD_STATE_ID_BUYER, F -> SHP_ADD_STATE_ID_SELLER, G -> SHP_LEAD_TIME_NATURAL_DAYS

    # Calculate the denominator first (sum of SHP for each SIT_SITE_ID, Year, Month group)
    denominator = df.groupby(['SIT_SITE_ID', year_col, month_col])['SHP'].transform('sum')

    # Calculate the numerator (sum of SHP for each group defined by all relevant columns)
    numerator = df.groupby(['SIT_SITE_ID', year_col, month_col, 'SHP_PICKING_TYPE_ID', 'SHP_ADD_STATE_ID_BUYER', 'SHP_ADD_STATE_ID_SELLER', 'SHP_LEAD_TIME_NATURAL_DAYS'])['SHP'].transform('sum')

    # Calculate SHARE_PT, handling division by zero
    df['SHARE_PT'] = numerator / denominator
    df['SHARE_PT'] = df['SHARE_PT'].fillna(0) # Replace NaN resulting from division by zero with 0


    # Calculate column SPEED_PT (K) based on the provided formula
    # H2/SUMIFS($H:$H;$A:$A;$A2;$B:$B;$B2;$C:$C;$C2;$D:$D;D2)
    # H -> SHP, A -> SIT_SITE_ID, B -> YEAR_FV/HT/C, C -> MONTH_FV/HT/C, D -> SHP_PICKING_TYPE_ID
    speed_pt_denominator = df.groupby(['SIT_SITE_ID', year_col, month_col, 'SHP_PICKING_TYPE_ID'])['SHP'].transform('sum')
    df['SPEED_PT'] = df['SHP'] / speed_pt_denominator
    df['SPEED_PT'] = df['SPEED_PT'].fillna(0) # Replace NaN resulting from division by zero with 0


    # Calculate column SPEED_ALL_NET (L) based on the provided formula
    # K2*SUMIFS($J:$J;$A:$A;$A2;$B:$B;$B2;$C:$C;$C2;$D:$D;$D2)
    # J -> SHARE_PT, A -> SIT_SITE_ID, B -> YEAR_FV/HT/C, C -> MONTH_FV/HT/C, D -> SHP_PICKING_TYPE_ID
    speed_all_net_sumifs = df.groupby(['SIT_SITE_ID', year_col, month_col, 'SHP_PICKING_TYPE_ID'])['SHARE_PT'].transform('sum')
    df['SPEED_ALL_NET'] = df['SPEED_PT'] * speed_all_net_sumifs


    # print("\nDataFrame columns after calculating new columns:")
    # print(df.columns)
    # Remove the display of the first 5 rows after calculating new columns
    # display(df.head())


    # Reorder columns
    if info_type == 'First Visit':
        ordered_columns = ['SIT_SITE_ID', 'YEAR_FV', 'MONTH_FV', 'SHP_PICKING_TYPE_ID', 'SHP_ADD_STATE_ID_BUYER', 'SHP_ADD_STATE_ID_SELLER', 'SHP_LEAD_TIME_NATURAL_DAYS', 'SHP', 'SIS', 'SHARE_PT', 'SPEED_PT', 'SPEED_ALL_NET']
    elif info_type == 'Handling':
         ordered_columns = ['SIT_SITE_ID', 'YEAR_HT', 'MONTH_HT', 'SHP_PICKING_TYPE_ID', 'SHP_ADD_STATE_ID_BUYER', 'SHP_ADD_STATE_ID_SELLER', 'SHP_LEAD_TIME_NATURAL_DAYS', 'SHP', 'SIS', 'SHARE_PT', 'SPEED_PT', 'SPEED_ALL_NET']
    elif info_type == 'Created':
        ordered_columns = ['SIT_SITE_ID', 'YEAR_C', 'MONTH_C', 'SHP_PICKING_TYPE_ID', 'SHP_ADD_STATE_ID_BUYER', 'SHP_ADD_STATE_ID_SELLER', 'SHP_LEAD_TIME_NATURAL_DAYS', 'SHP', 'SIS', 'SHARE_PT', 'SPEED_PT', 'SPEED_ALL_NET']

    # Ensure all required columns exist in the DataFrame before reordering
    missing_columns = [col for col in ordered_columns if col not in df.columns]
    if missing_columns:
        print(f"Warning: The following columns are missing from the DataFrame and cannot be reordered: {missing_columns}")
        # Proceed with reordering only the existing columns
        ordered_columns = [col for col in ordered_columns if col in df.columns]


    df = df[ordered_columns]

    # print("\nDataFrame columns after reordering:")
    # print(df.columns)
    # Remove the display of the first 5 rows after reordering as requested by the user
    # display(df.head())


    # Guardar resultados a un archivo Excel
    df.to_excel("info.xlsx", index=False, sheet_name="Info Real")
    # print("Resultados guardados en 'info.xlsx' con la hoja 'Info Real'.")

# Ejecutar el menú y la consulta
country, info_type, year, month = menu_selection()
execute_query(country, info_type, year, month)


**********************************************************************
POR FAVOR, PROPORCIONE LOS SIGUIENTES PARÁMETROS PARA OBTENER LA INFORMACIÓN HISTORICA/REAL:
**********************************************************************

SELECCIONE EL PAÍS (SIT_SITE_ID):
1. MCO
2. MEC
3. MLA
4. MLB
5. MLC
6. MLM
7. MLU
8. MPE
Ingrese el número del país: 1

SELECCIONE EL TIPO DE INFORMACIÓN:
1. First Visit
2. Handling
3. Created
Ingrese el número del tipo de información: 1

INGRESE EL AÑO (2023 - 2025): 1
Fecha inválida, debe ser entre 2023 y el año actual. Inténtalo de nuevo.

INGRESE EL AÑO (2023 - 2025): 2025

INGRESE EL MES (1 - 12): 1
Executing query with:
  Country: MCO
  Info Type: First Visit
  Date Column: SHP_DATE_FIRST_VISIT_ID_LTZ
  Year: 2025
  Month: 1
Number of rows returned by the query: 10568
Data type of 'SIS' column: object
